# Refactoring object based Strategy to function based Strategy
## Strategy Pattern
The Strategy pattern is summarized like this in the Design Patterns book:
> Define a family of algorithms, encapsulate each one, and make them interchangeable. Strategy lets the algorithm vary independently from clients that use it.

A clear example of Strategy applied in the e-commerce domain is computing discounts to orders according to the attributes of the customer or inspection of the ordered items.
Consider an online store with these discount rules:
- customers with 1000 or more fidelity points get a global 5% discount per order;
- a 10% discount is applied to each line item with 20 or more units in the same order;
- orders with at least 10 distinct items get a 7% global discount.

> ### Context
> Provides a service by delegating some computation to interchangeable components that implement alternative algorithms. In an e-commerce example, the context is an Order, which is configured to apply a promotional discount according to one of several algorithms.
> ### Strategy
> The interface common to the components that implement the different algorithms. In an ecommerce example, this role is played by an abstract class called Promotion.
> ### Concrete Strategy
> One of the concrete subclasses of Strategy. FidelityPromo, BulkPromo and Large OrderPromo are the three concrete strategies implemented.


## Object-oriented Strategy

In [2]:
"""Implementation of Order class with pluggable discount strategies"""

from abc import ABC, abstractmethod
from collections import namedtuple


Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

    
class Order:  # Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

    
class Promotion(ABC):  # The Strategy - an ABC
    
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # first concrete strategy
    """5% discount for customers with 1000 or more fidelity points"""
    def discount(self, order):
        return order.total() * 0.5 if order.customer.fidelity >= 1000 else 0

    
class BulkItemPromo(Promotion):  # second concrete strategy
    """10% discount for each LineItem with 20 or more units"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount
    

class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0


In [10]:
joe = Customer('John Doe', 0)


In [5]:
ann = Customer('Ann Smith', 1100)

In [6]:
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]

In [7]:
print(Order(joe, cart, FidelityPromo()))
print(Order(ann, cart, FidelityPromo()))

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 21.00>


In [8]:
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
print(Order(joe, banana_cart, BulkItemPromo()))


<Order total: 30.00 due: 28.50>


In [9]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print(Order(joe, long_order, LargeOrderPromo()))
print(Order(joe, cart, LargeOrderPromo()))

<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 42.00>


In [11]:
"""All test cases"""
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]
print(Order(joe, cart, FidelityPromo()))
print(Order(ann, cart, FidelityPromo()))

banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
print(Order(joe, banana_cart, BulkItemPromo()))

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print(Order(joe, long_order, LargeOrderPromo()))
print(Order(joe, cart, LargeOrderPromo()))

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 21.00>
<Order total: 30.00 due: 28.50>
<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 42.00>


## Function-based Strategy